In [338]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import math
#import Kmeans algorithm from sci kit learn library
from sklearn.cluster import KMeans
#import scaler package for standardizing values
from sklearn.preprocessing import StandardScaler
import random
% matplotlib inline

# Receive user input

In [339]:
demo_raw = [4,0,0,0,3,3,4,7,6,3,0,0]

# Import All Files

In [340]:
def import_tables():
    demo_lookup = pd.read_csv('lookup_matrix.csv')
    grades_lookup = pd.read_csv('grades_lookup.csv')
    demo_lookup = demo_lookup.drop('Index ', axis = 1)
    total_output = pd.read_csv("Design thinking clusters flat file v4.csv")
    flat = pd.read_csv('Full_Flat_File.csv')
    CPI = pd.read_csv('CPI.csv')
    
    return demo_lookup, grades_lookup, demo_lookup, total_output, flat, CPI

In [341]:
demo_lookup, grades_lookup, demo_lookup, total_output, flat, CPI = import_tables()

# Clean Up

In [342]:
#Function that Turns input into readable values
def input_convert(demo_raw):
    demo_input = []
    i = 0
    for value in demo_raw:
        value = demo_lookup.iloc[value, i]
        demo_input.append(value)
        i += 1 
    return demo_input 
    

In [343]:
demo_input = input_convert(demo_raw)

In [344]:
#function that converts all values to list for clustering 
def values_convert(demo_input):
    
    #initiate empty list
    demo_input_clean = []
    #year equal to input
    year = demo_input[0]

    #program equal to input
    prog = demo_input[1]
    #provide random student number
    std_no = random.randint(1,1000)
    #take the mean of the range for co-op salalry
    adj_ann_salary_hrlyfirst = (int(demo_input[2].split("-")[0]) + int(demo_input[2].split("-")[1]))/2

    adj_ann_salary_hrlylast = (int(demo_input[3].split("-")[0]) + int(demo_input[3].split("-")[1]))/2
    
    #take raw perf eval
    perf_eval_convfirst = int(demo_input[4])

    perf_eval_convlast = int(demo_input[5])
    #student eval is the average of the two perf eval 
    stud_eval_convfirst= (perf_eval_convfirst + perf_eval_convlast)/2
    stud_eval_convlast= (perf_eval_convfirst + perf_eval_convlast)/2
    #take raw 
    no_terms = int(demo_input[6])

    # take average of range of uni marks 
    if demo_input[7] == '<50':
        u_avg = 50
    elif demo_input[7] == '95+':
        u_avg = 97.5
    else:
        u_avg = (int(demo_input[7].split("-")[0]) + int(demo_input[7].split("-")[1]))/2
    # do same for high school 
    if demo_input[8] == '<50':
        h_avg = 50
    elif demo_input[8] == '95+':
        h_avg = 97.5
    else:
        h_avg = (int(demo_input[8].split("-")[0]) + int(demo_input[8].split("-")[1]))/2
        
    # of terms is the years multiplied by 2
    no_acad_terms = demo_input[9] * 2
    gen_ind = demo_input[10]
    stem_ind = demo_input[11]

    # lookup the average grade of that program, for the expected average
    exp_u_avg = pd.merge(pd.DataFrame(demo_input).transpose(), grades_lookup, 'inner',
                     left_on = 1, right_on = 'acad_plan_descr')['course_average_grade5'][0]
    #divide user average by expected average to get ratio
    
    c_avg_r = u_avg / exp_u_avg

    #elective equal to cor for simplicity
    e_avg_r = c_avg_r
    
    #take average high school to uni ratio
    avg_u_diff = (total_output['Core_Average'] /total_output['Entrace_Average_Ratio']).mean()

    #apply to hs average 
    h_avg_r = c_avg_r * avg_u_diff
    
    #create list
    demo_input_clean.extend([year, std_no, adj_ann_salary_hrlyfirst, adj_ann_salary_hrlylast, perf_eval_convfirst, perf_eval_convlast, no_terms,
                             stud_eval_convfirst, stud_eval_convlast, c_avg_r, e_avg_r, no_acad_terms, h_avg_r, gen_ind, stem_ind])

    return demo_input_clean, std_no
    

In [345]:
demo_input_clean, std_no = values_convert(demo_input)

In [347]:
#function takes clean input, and converts to final dataframe 
def cluster_input_convert(demo_input_clean):
    demo_list = ['calendar_year','STUDENT_KEY','adj_ann_salary_hrlyfirst',
                 'adj_ann_salary_hrlylast','perf_eval_convfirst','perf_eval_convlast',
         'Uw Co Job Eval Stufirst',
         'Uw Co Job Eval Stulast',
         'Uw Co Wt Session',
         'Core_Average',
         'Elective_Average',
         'Academic_Terms',
         'Entrace_Average_Ratio',
         'Female_Ind', 'STEM_Ind']

    demo_df = pd.DataFrame(demo_input_clean).transpose()
    demo_df.columns = demo_list           
    return demo_df 

In [348]:
demo_df = cluster_input_convert(demo_input_clean)

In [349]:
def salary_adj(demo_df):
    demo_df = pd.merge(demo_df, CPI, 'inner', 'calendar_year')
    demo_df['adj_ann_salary_hrlyfirst'] = (demo_df['adj_ann_salary_hrlyfirst'] * 133.4) /demo_df['CPI']
    demo_df['adj_ann_salary_hrlylast'] = (demo_df['adj_ann_salary_hrlylast'] * 133.4) /demo_df['CPI']
    demo_df_adj = demo_df.drop(['calendar_year', 'CPI'], axis = 1)
    return demo_df_adj

In [350]:
demo_df_adj = salary_adj(demo_df)

In [351]:
def cluster_input_final(demo_df_adj):
    clust_list = ['STUDENT_KEY','adj_ann_salary_hrlyfirst',
 'adj_ann_salary_hrlylast',
 'perf_eval_convfirst',
 'perf_eval_convlast',
 'Uw Co Job Eval Stufirst',
 'Uw Co Job Eval Stulast',
 'Uw Co Wt Session',
 'Core_Average',
 'Elective_Average',
 'Academic_Terms',
 'Entrace_Average_Ratio',
 'Female_Ind', 'STEM_Ind']
    flat_final = flat[clust_list]
    flat_final = flat_final.append(demo_df_adj).reset_index(drop = True)
    attr_list = list(flat_final.drop(['STUDENT_KEY'], axis =1))
    full_flat_no_na = flat_final.dropna()
    return attr_list, full_flat_no_na
    

In [352]:
attr_list, df = cluster_input_final(demo_df_adj)

# Clustering Variable Combinations

In [353]:
#Function that takes in a subset of data, number of clusters,  and scales the data, runs kmeans, and returns dataframe & centroids 
def cluster(n, col_list, df): 
    #perform analysis on copy of original dataset 
    df_copy = df.copy(deep = True)
    #create subset of dataset based on list 
    c_df = df_copy[col_list]
    #initiate kmeans based on number of clusters chosen 
    km = KMeans(n_clusters= n)
    #initiate scaler
    scaler = StandardScaler()
    #fit scaler to data 
    scaled = scaler.fit(c_df)
    #transform the data
    df_scale = pd.DataFrame(scaled.transform(c_df), columns = list(c_df))
    #fit kmeans to the data
    km.fit(df_scale)
    #create column in dataframe with cluster numbrs 
    c_df['cluster'] = km.labels_
    return c_df   

# Cluster

In [354]:
def user_cluster():
    clustered = cluster(6, attr_list, df)
    centroids = clustered.groupby('cluster').mean()
    clustered['STUDENT_KEY'] = df['STUDENT_KEY']
    clustered['cluster'] = clustered['cluster'] +1
    final_df = centroids.reset_index()
    final_df['cluster'] = final_df['cluster'] +1
    final_df['metric'] = final_df['adj_ann_salary_hrlyfirst']+ final_df['adj_ann_salary_hrlylast'] + final_df['perf_eval_convfirst']+ final_df['perf_eval_convlast']+final_df['Core_Average'] +final_df['Uw Co Wt Session']
    final_df = final_df.sort_values('metric', ascending = False)
    clust_names = ['The Gold Standards', 'The High Potentials', 'The Experience Seekers', 'Intrinsic Go Getter', 'The Participators', 'The Low Performers']
    clust_number = [4,6,2,5,1,3 ]
    final_df['clust_names']= clust_names
    final_df['clust_num'] = clust_number
    user_cluster = (clustered[clustered['STUDENT_KEY'] == std_no]['cluster'])[clustered[clustered['STUDENT_KEY'] == std_no].index[0]]
    clust_name = final_df[final_df['cluster'] == user_cluster].reset_index()['clust_names'][0]
    clust_num = final_df[final_df['cluster'] == user_cluster].reset_index()['clust_num'][0]
    return clust_name, clust_num, final_df

In [355]:
name, number, centroids = user_cluster()
print ("Your cluster number is %s!" % number)
print ("Your cluster is the %s!" % name)


Your cluster number is 1!
Your cluster is the The Participators!


In [356]:

#This combination with 6 clusters proved to be the most meaningful. 4 clusters did not tell the entire story, and it was difficult
#to make clear distinctionss between all eight clusters. 
#This analysis made it clear that
#1) there are stem females who earn very little but do well in school
#2) the group of stem males that does well across the board is male 
#''''